In [30]:
import pandas
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

In [32]:
data_path = '/data/linghui/black-friday/'
train_file = data_path + 'train.csv'

row_data = pd.read_csv(train_file, sep=',')
row_data = row_data.fillna(0)
row_data.columns = ['user_id','product_id','gender','age','occupation','city','stay_years','martial_status','cate_1','cate_2','cate_3','price']
lbe_feats = ['user_id','product_id','gender','age','occupation','city','stay_years','martial_status','cate_1','cate_2','cate_3']
for feat in lbe_feats:
    lbe = LabelEncoder()
    row_data[feat] = lbe.fit_transform(row_data[feat])
row_data

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price
0,0,672,0,0,10,0,2,0,2,0,0,8370
1,0,2376,0,0,10,0,2,0,0,5,11,15200
2,0,852,0,0,10,0,2,0,11,0,0,1422
3,0,828,0,0,10,0,2,0,11,13,0,1057
4,1,2734,1,6,16,2,4,0,7,0,0,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,5883,3567,1,5,13,1,1,1,19,0,0,368
550064,5885,3568,0,2,1,2,3,0,19,0,0,371
550065,5886,3568,0,2,15,1,4,1,19,0,0,137
550066,5888,3568,0,6,1,2,2,0,19,0,0,365


In [147]:
row_data.price.max()

23961

In [97]:
col_null = row_data.isnull().sum(axis=0)
col_null

user_id           0
product_id        0
gender            0
age               0
occupation        0
city              0
stay_years        0
martial_status    0
cate_1            0
cate_2            0
cate_3            0
price             0
dtype: int64

In [132]:
def create_black_friday_data(data):
    data = data.sort_values(['user_id'])
    num = len(data.user_id.value_counts())
    test_pd = pd.DataFrame(columns=data.columns)
    for i in range(num):
        user_select_group = data.loc[data['user_id'] == i]
        user_pd = user_select_group.iloc[-1]
        test_pd = test_pd.append(user_pd)
    drop_index = test_pd.index.to_list()
    train_pd = data[~data.index.isin(drop_index)]
    return train_pd,test_pd

train_data , test_data = create_black_friday_data(row_data)

In [35]:
train_data.user_id.value_counts()
# test_data.user_id.value_counts()

1634    1025
4166     978
1885     897
1142     861
863      822
        ... 
2052       6
5253       6
2615       6
5467       6
686        5
Name: user_id, Length: 5891, dtype: int64

In [133]:
test_data

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price
1,0,2376,0,0,10,0,2,0,0,5,11,15200
116853,1,1972,1,6,16,2,4,0,5,7,0,20434
467670,2,1079,1,2,15,0,3,0,0,1,3,3938
8,3,3604,1,4,7,1,2,1,0,15,0,15686
78152,4,286,1,2,20,0,1,1,7,0,0,5894
...,...,...,...,...,...,...,...,...,...,...,...,...
233683,5886,1793,0,2,15,1,4,1,1,3,0,10117
78118,5887,3101,0,4,1,2,4,0,7,0,0,7924
506583,5888,1862,0,6,1,2,2,0,7,13,0,7815
78130,5889,1032,0,4,0,1,4,1,12,15,0,946


In [40]:
user_data = row_data.loc[:,['user_id','gender','age','occupation','city','stay_years','martial_status']]
user_data = user_data.drop_duplicates()
user_data = user_data.reset_index(drop=True)
user_data

,user_id,gender,age,occupation,city,stay_years,martial_status
0,0,0,0,10,0,2,0
1,1,1,6,16,2,4,0
2,2,1,2,15,0,3,0
3,3,1,4,7,1,2,1
4,4,1,2,20,0,1,1
...,...,...,...,...,...,...,...
5886,4474,0,2,4,2,0,0
5887,4749,1,1,12,2,2,0
5888,4008,1,3,17,2,3,0
5889,5253,1,2,7,0,0,0


In [41]:
item_data = row_data.loc[:,['product_id','cate_1','cate_2','cate_3','price']]
item_data = item_data.drop_duplicates()
item_data = item_data.reset_index(drop=True)
item_data

,product_id,cate_1,cate_2,cate_3,price
0,672,2,0,0,8370
1,2376,0,5,11,15200
2,852,11,0,0,1422
3,828,11,13,0,1057
4,2734,7,0,0,7969
...,...,...,...,...,...
473628,3567,19,0,0,123
473629,3566,19,0,0,591
473630,3568,19,0,0,125
473631,3567,19,0,0,374


In [53]:
item_num = item_data['product_id'].nunique()
item_price_dict = {}
for i in range(item_num):
    item_temp = item_data.loc[item_data['product_id'] == i]
    item_price = item_temp['price'].unique()
    item_price_dict[i] = item_price

In [63]:
price_list = item_price_dict[0]
a = np.random.choice(price_list, size=1, replace=True)
a[0]

10947

In [116]:
def neg_sample(u_data, item_ids,neg_rate_train=1,neg_rate_test = 49):
    print('start neg sample')
    neg_data_train = []
    neg_data_test = []
    for user_id, hist in tqdm(u_data.groupby('user_id')):
        purchased_item_list = hist['product_id'].tolist()
        candidate_set = list(set(item_ids) - set(purchased_item_list))
        neg_list_train = np.random.choice(candidate_set, size=(len(purchased_item_list)-1) * neg_rate_train, replace=True)
        neg_list_test = np.random.choice(candidate_set, size=neg_rate_test, replace=True)
        for id in neg_list_train:
            list_train = create_neg_data(user_id,id,item_price_dict)
            neg_data_train.append(list_train)
        for id in neg_list_test:
            list_test = create_neg_data(user_id,id,item_price_dict)
            neg_data_test.append(list_test)
    u_data_neg_train = pd.DataFrame(neg_data_train)
    u_data_neg_test = pd.DataFrame(neg_data_test)
    # print(u_data_neg_train)
    # print(u_data_neg_test)
    u_data_neg_train.columns = ['user_id', 'product_id', 'price','label']
    u_data_neg_test.columns = ['user_id', 'product_id', 'price','label']
    return u_data_neg_train,u_data_neg_test

def create_neg_data(user_id,item_id,item_price_dict):
    price_list = item_price_dict[item_id]
    price = np.random.choice(price_list, size=1, replace=True)
    price = price[0]
    result = [user_id,item_id,price,0]
    return result

In [117]:
item_ids = item_data['product_id'].unique()
neg_train , neg_test = neg_sample(row_data,item_ids , 1, 49)
neg_train = pd.merge(neg_train, user_data , on=['user_id'],how = 'left')
neg_train = pd.merge(neg_train, item_data,  on=['product_id','price'],how = 'left')

neg_test = pd.merge(neg_test, user_data , on=['user_id'],how = 'left')
neg_test = pd.merge(neg_test, item_data,  on=['product_id','price'],how = 'left')

start neg sample


100%|██████████| 5891/5891 [00:29<00:00, 202.99it/s]


In [118]:
neg_train

,user_id,product_id,price,label,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3
0,0,1697,1669,0,0,0,10,0,2,0,10,0,0
1,0,2103,15690,0,0,0,10,0,2,0,0,0,0
2,0,3273,15790,0,0,0,10,0,2,0,0,7,14
3,0,2532,1738,0,0,0,10,0,2,0,10,0,0
4,0,3193,7898,0,0,0,10,0,2,0,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544172,5890,2207,12570,0,1,2,6,1,2,0,5,0,0
544173,5890,2843,7799,0,1,2,6,1,2,0,7,0,0
544174,5890,1722,6974,0,1,2,6,1,2,0,1,3,6
544175,5890,1910,19391,0,1,2,6,1,2,0,0,5,5


In [120]:
train_data['label'] = 1
train_data

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price,label
0,0,672,0,0,10,0,2,0,2,0,0,8370,1
467664,0,2098,0,0,10,0,2,0,7,0,0,7953,1
467663,0,570,0,0,10,0,2,0,2,3,9,11051,1
428894,0,722,0,0,10,0,2,0,2,3,9,10572,1
428893,0,249,0,0,10,0,2,0,0,1,6,15416,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194557,5890,2844,1,2,6,1,2,0,6,0,0,21005,1
506600,5890,3290,1,2,6,1,2,0,4,17,0,6915,1
506599,5890,867,1,2,6,1,2,0,7,0,0,9868,1
506598,5890,3321,1,2,6,1,2,0,0,14,0,15358,1


In [123]:
train_data_final = pd.concat([train_data,neg_train])

In [130]:
train_data_final

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price,label
0,0,672,0,0,10,0,2,0,2,0,0,8370,1
467664,0,2098,0,0,10,0,2,0,7,0,0,7953,1
467663,0,570,0,0,10,0,2,0,2,3,9,11051,1
428894,0,722,0,0,10,0,2,0,2,3,9,10572,1
428893,0,249,0,0,10,0,2,0,0,1,6,15416,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544172,5890,2207,1,2,6,1,2,0,5,0,0,12570,0
544173,5890,2843,1,2,6,1,2,0,7,0,0,7799,0
544174,5890,1722,1,2,6,1,2,0,1,3,6,6974,0
544175,5890,1910,1,2,6,1,2,0,0,5,5,19391,0


In [134]:
test_data['label'] = 1
test_data_final = pd.concat([test_data,neg_test])

In [148]:
test_data_final

,user_id,product_id,gender,age,occupation,city,stay_years,martial_status,cate_1,cate_2,cate_3,price,label
1,0,2376,0,0,10,0,2,0,0,5,11,15200,1
116853,1,1972,1,6,16,2,4,0,5,7,0,20434,1
467670,2,1079,1,2,15,0,3,0,0,1,3,3938,1
8,3,3604,1,4,7,1,2,1,0,15,0,15686,1
78152,4,286,1,2,20,0,1,1,7,0,0,5894,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288654,5890,1213,1,2,6,1,2,0,10,15,0,6110,0
288655,5890,3094,1,2,6,1,2,0,4,0,0,7095,0
288656,5890,1824,1,2,6,1,2,0,4,13,0,3709,0
288657,5890,3256,1,2,6,1,2,0,7,0,0,6010,0


In [138]:
saved_data_path = '/data/linghui/black-friday/processed_data/'
train_data_final.to_csv(saved_data_path + 'train_data_liuyi.csv')
test_data_final.to_csv(saved_data_path + 'test_data_liuyi.csv')

In [139]:
import sys
deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFEFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

DeepCTR version 0.9.1 detected. Your version is 0.9.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


In [140]:
merge_data = pd.concat([train_data_final,test_data_final])

In [141]:
sparse_features = ['user_id','product_id','gender','age','occupation','city','stay_years','martial_status','cate_1','cate_2','cate_3','price','label']

fixlen_feature_columns = [SparseFeat(feat, merge_data[feat].nunique(), embedding_dim=4)
								  for feat in sparse_features]

In [146]:
feat_dict = {}
for feat in sparse_features:
    feat_dict[feat] = merge_data[feat].nunique()
feat_dict

{'user_id': 5891,
 'product_id': 3631,
 'gender': 2,
 'age': 7,
 'occupation': 21,
 'city': 3,
 'stay_years': 5,
 'martial_status': 2,
 'cate_1': 20,
 'cate_2': 18,
 'cate_3': 16,
 'price': 18105,
 'label': 2}